# Food Price Data Source

[WFP Food Prices Kenya Dataset](https://data.humdata.org/dataset/wfp-food-prices-for-kenya)

# Rainfall Data Source
[WFP Rainfall Kenya Dataset](https://data.humdata.org/dataset/ken-rainfall-subnational)

# -------------------------------------------------------------------------------------------






# DATA CLEANING

In [4]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('FoodPricePrediction').master('local[*]').getOrCreate()

spark.sparkContext.appName

25/04/15 03:11:09 WARN Utils: Your hostname, codespaces-ebd91c resolves to a loopback address: 127.0.0.1; using 10.0.1.23 instead (on interface eth0)
25/04/15 03:11:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/15 03:11:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'FoodPricePrediction'

25/04/15 03:11:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
data = spark.read.csv("data/wfp_food_prices_ken_data.csv",inferSchema=True,header=True)
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- region: string (nullable = true)
 |-- county: string (nullable = true)
 |-- market: string (nullable = true)
 |-- category: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- pricetype: string (nullable = true)
 |-- price: double (nullable = true)



In [7]:
data.show(5)

+---------+-------+--------+--------+------------------+-------------+-----+---------+------+
|     date| region|  county|  market|          category|    commodity| unit|pricetype| price|
+---------+-------+--------+--------+------------------+-------------+-----+---------+------+
|1/15/2006|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 16.13|
|1/15/2006|Eastern|   Kitui|   Kitui|cereals and tubers|      Sorghum|90 KG|Wholesale|1800.0|
|1/15/2006|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  39.0|
|1/15/2006|Eastern|Marsabit|Marsabit|cereals and tubers|Maize (white)|   KG|   Retail|  21.0|
|1/15/2006|Nairobi| Nairobi| Nairobi|cereals and tubers|        Bread|400 G|   Retail|  26.0|
+---------+-------+--------+--------+------------------+-------------+-----+---------+------+
only showing top 5 rows



In [8]:
rainfall = spark.read.csv("data/ken-rainfall-data.csv",inferSchema=True,header=True)
rainfall.printSchema()

root
 |-- date: string (nullable = true)
 |-- rainfall_mm: double (nullable = true)



In [9]:
rainfall.show(5)

+---------+-----------+
|     date|rainfall_mm|
+---------+-----------+
| 1/1/1981|       NULL|
|1/11/1981|       NULL|
|1/21/1981|       NULL|
| 2/1/1981|       NULL|
|2/11/1981|       NULL|
+---------+-----------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import to_date

# Convert date strings to DateType
data = data.withColumn("date", to_date("date", "M/d/yyyy"))
rainfall = rainfall.withColumn("date", to_date("date", "M/d/yyyy"))


In [11]:
from pyspark.sql import functions as F

# Split the 'date' column into 'month' and 'year'
data1 = data.withColumn('month', F.month('date')) \
                       .withColumn('year', F.year('date'))
data1 = data1.drop('date')

data1.show(5)


+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
| region|  county|  market|          category|    commodity| unit|pricetype| price|month|year|
+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 16.13|    1|2006|
|Eastern|   Kitui|   Kitui|cereals and tubers|      Sorghum|90 KG|Wholesale|1800.0|    1|2006|
|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  39.0|    1|2006|
|Eastern|Marsabit|Marsabit|cereals and tubers|Maize (white)|   KG|   Retail|  21.0|    1|2006|
|Nairobi| Nairobi| Nairobi|cereals and tubers|        Bread|400 G|   Retail|  26.0|    1|2006|
+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import col, sum

# Count nulls in each column for 'data' DataFrame
null_counts = data.select([sum(col(c).isNull().cast("int")).alias(c) for c in data.columns])
null_counts.show()

# Count nulls in each column for 'rainfall' DataFrame
rainfall_null_counts = rainfall.select([sum(col(c).isNull().cast("int")).alias(c) for c in rainfall.columns])
rainfall_null_counts.show()

+----+------+------+------+--------+---------+----+---------+-----+
|date|region|county|market|category|commodity|unit|pricetype|price|
+----+------+------+------+--------+---------+----+---------+-----+
|   0|    44|    44|     0|       0|        0|   0|        0|    0|
+----+------+------+------+--------+---------+----+---------+-----+

+----+-----------+
|date|rainfall_mm|
+----+-----------+
|   0|        584|
+----+-----------+



In [13]:
data_clean = data.dropna()
rainfall_clean = rainfall.dropna()

In [14]:
data_clean.count(), len(data.columns)

(12702, 9)

In [15]:
rainfall_clean.count(), len(rainfall.columns)

(115705, 2)

In [16]:
data_clean.show(5)

+----------+-------+--------+--------+------------------+-------------+-----+---------+------+
|      date| region|  county|  market|          category|    commodity| unit|pricetype| price|
+----------+-------+--------+--------+------------------+-------------+-----+---------+------+
|2006-01-15|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 16.13|
|2006-01-15|Eastern|   Kitui|   Kitui|cereals and tubers|      Sorghum|90 KG|Wholesale|1800.0|
|2006-01-15|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  39.0|
|2006-01-15|Eastern|Marsabit|Marsabit|cereals and tubers|Maize (white)|   KG|   Retail|  21.0|
|2006-01-15|Nairobi| Nairobi| Nairobi|cereals and tubers|        Bread|400 G|   Retail|  26.0|
+----------+-------+--------+--------+------------------+-------------+-----+---------+------+
only showing top 5 rows



In [17]:
rainfall_clean.show(5)

+----------+-----------+
|      date|rainfall_mm|
+----------+-----------+
|1981-03-21|   266.3542|
|1981-04-01|     360.75|
|1981-04-11|      542.5|
|1981-04-21|   608.1042|
|1981-05-01|   767.2083|
+----------+-----------+
only showing top 5 rows



In [18]:
from pyspark.sql import functions as F

# Split the 'date' column into 'month' and 'year'
data_clean = data_clean.withColumn('month', F.month('date')) \
                       .withColumn('year', F.year('date'))

# Drop the 'date' column
data_clean = data_clean.drop('date')

# Filter years between 2014 and 2024 (inclusive)
data_clean = data_clean.filter((data_clean.year >= 2014) & (data_clean.year <= 2024))

# Sort by year and month
data_clean = data_clean.orderBy("year", "month")

data_clean.show(5)

+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
| region|  county|  market|          category|    commodity| unit|pricetype| price|month|year|
+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 38.44|    1|2014|
|  Coast| Mombasa| Mombasa|   pulses and nuts|        Beans|   KG|Wholesale| 79.99|    1|2014|
|  Coast| Mombasa| Mombasa|   pulses and nuts|  Beans (dry)|90 KG|Wholesale|5738.0|    1|2014|
|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  74.0|    1|2014|
|Eastern|Marsabit|Marsabit|cereals and tubers|Maize (white)|   KG|   Retail| 53.36|    1|2014|
+-------+--------+--------+------------------+-------------+-----+---------+------+-----+----+
only showing top 5 rows



In [19]:
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, round

# Split the 'date' column into 'month' and 'year'
rainfall_clean = rainfall_clean.withColumn('month', F.month('date')) \
                               .withColumn('year', F.year('date'))

# Drop the 'date' column
rainfall_clean = rainfall_clean.drop('date')

# Group by year and month, and calculate average rainfall rounded to 2 decimal places
rainfall_clean = rainfall_clean.groupBy("year", "month").agg(
    round(avg("rainfall_mm"), 2).alias("avg_rainfall_mm")
)

# Filter years between 2014 and 2024 (inclusive)
rainfall_clean = rainfall_clean.filter((rainfall_clean.year >= 2014) & (rainfall_clean.year <= 2024))

# Sort by year and month
rainfall_clean = rainfall_clean.orderBy("year", "month")

# Show first 5 rows
rainfall_clean.show(5)

+----+-----+---------------+
|year|month|avg_rainfall_mm|
+----+-----+---------------+
|2014|    1|         259.33|
|2014|    2|         201.62|
|2014|    3|         184.92|
|2014|    4|         269.88|
|2014|    5|         339.76|
+----+-----+---------------+
only showing top 5 rows



In [20]:
# Join market price data with rainfall data on year and month
food_price_data = data_clean.join(
    rainfall_clean,
    on=["year", "month"],  # Join keys
    how="left"
)

# Show sample of the joined result
food_price_data.show(5)

+----+-----+-------+--------+--------+------------------+-------------+-----+---------+------+---------------+
|year|month| region|  county|  market|          category|    commodity| unit|pricetype| price|avg_rainfall_mm|
+----+-----+-------+--------+--------+------------------+-------------+-----+---------+------+---------------+
|2014|    1|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 38.44|         259.33|
|2014|    1|  Coast| Mombasa| Mombasa|   pulses and nuts|        Beans|   KG|Wholesale| 79.99|         259.33|
|2014|    1|  Coast| Mombasa| Mombasa|   pulses and nuts|  Beans (dry)|90 KG|Wholesale|5738.0|         259.33|
|2014|    1|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  74.0|         259.33|
|2014|    1|Eastern|Marsabit|Marsabit|cereals and tubers|Maize (white)|   KG|   Retail| 53.36|         259.33|
+----+-----+-------+--------+--------+------------------+-------------+-----+---------+------+---------------+
o

In [21]:
from pyspark.sql.functions import col, sum

# Count nulls
nulls = food_price_data.select([sum(col(c).isNull().cast("int")).alias(c) for c in food_price_data.columns])
nulls.show()

+----+-----+------+------+------+--------+---------+----+---------+-----+---------------+
|year|month|region|county|market|category|commodity|unit|pricetype|price|avg_rainfall_mm|
+----+-----+------+------+------+--------+---------+----+---------+-----+---------------+
|   0|    0|     0|     0|     0|       0|        0|   0|        0|    0|              0|
+----+-----+------+------+------+--------+---------+----+---------+-----+---------------+



In [22]:
for column in food_price_data.columns:
    print(f"\nUnique values for column: {column}")
    food_price_data.select(column).distinct().show(truncate=False)


Unique values for column: year


+----+
|year|
+----+
|2018|
|2015|
|2023|
|2022|
|2014|
|2019|
|2020|
|2016|
|2024|
|2017|
|2021|
+----+


Unique values for column: month
+-----+
|month|
+-----+
|12   |
|1    |
|6    |
|3    |
|5    |
|9    |
|4    |
|8    |
|7    |
|10   |
|11   |
|2    |
+-----+


Unique values for column: region
+-------------+
|region       |
+-------------+
|Rift Valley  |
|Eastern      |
|North Eastern|
|Nyanza       |
|Coast        |
|Central      |
|Nairobi      |
+-------------+


Unique values for column: county
+-----------+
|county     |
+-----------+
|Uasin Gishu|
|Nakuru     |
|Mandera    |
|Kisumu     |
|Marsabit   |
|Wajir      |
|Kajiado    |
|Turkana    |
|Mombasa    |
|Kwale      |
|Makueni    |
|Meru South |
|Garissa    |
|Nairobi    |
|Isiolo     |
|Kitui      |
|Kilifi     |
|Baringo    |
|West Pokot |
|Nyeri      |
+-----------+
only showing top 20 rows


Unique values for column: market
+-------------------------------+
|market                         |
+----------------------

In [23]:
from pyspark.sql.functions import udf, col, round as spark_round, log1p
from pyspark.sql.types import DoubleType

# List of units to drop
units_to_drop = ["Unit", "Bunch", "Head"]
food_price_data = food_price_data.filter(~col("unit").isin(units_to_drop))

# Define conversion factors
unit_conversion = {
    '400 G': 2.5,
    '64 KG': 1/64,
    'L': 1,
    '200 G': 5,
    '50 KG': 1/50,
    '13 KG': 1/13,
    '90 KG': 1/90,
    '200 ML': 5,
    '126 KG': 1/126,
    'KG': 1,
    '500 ML': 2
}

# Create UDF
def normalize_price(price, unit):
    factor = unit_conversion.get(unit, 1)
    return price * factor

normalize_price_udf = udf(normalize_price, DoubleType())

# Apply normalization
food_price_data = food_price_data.withColumn("normalized_price", normalize_price_udf(col("price"), col("unit")))

# Round to 2 decimal places
food_price_data = food_price_data.withColumn("normalized_price", spark_round(col("normalized_price"), 2))

# log transform to reduce skewness
food_price_data = food_price_data.withColumn("log_normalized_price", log1p(col("normalized_price")))

# Show first 5 rows
food_price_data.select("price", "unit", "normalized_price", "log_normalized_price").show(5, truncate=False)

+------+-----+----------------+--------------------+
|price |unit |normalized_price|log_normalized_price|
+------+-----+----------------+--------------------+
|38.44 |KG   |38.44           |3.6747805297344347  |
|79.99 |KG   |79.99           |4.3943256902608985  |
|5738.0|90 KG|63.76           |4.170688128809434   |
|74.0  |KG   |74.0            |4.31748811353631    |
|53.36 |KG   |53.36           |3.995628589282943   |
+------+-----+----------------+--------------------+
only showing top 5 rows



In [24]:
food_price_data.show(4)

+----+-----+-------+-------+-------+------------------+-----------+-----+---------+------+---------------+----------------+--------------------+
|year|month| region| county| market|          category|  commodity| unit|pricetype| price|avg_rainfall_mm|normalized_price|log_normalized_price|
+----+-----+-------+-------+-------+------------------+-----------+-----+---------+------+---------------+----------------+--------------------+
|2014|    1|  Coast|Mombasa|Mombasa|cereals and tubers|      Maize|   KG|Wholesale| 38.44|         259.33|           38.44|  3.6747805297344347|
|2014|    1|  Coast|Mombasa|Mombasa|   pulses and nuts|      Beans|   KG|Wholesale| 79.99|         259.33|           79.99|  4.3943256902608985|
|2014|    1|  Coast|Mombasa|Mombasa|   pulses and nuts|Beans (dry)|90 KG|Wholesale|5738.0|         259.33|           63.76|   4.170688128809434|
|2014|    1|Eastern|  Kitui|  Kitui|   pulses and nuts|Beans (dry)|   KG|   Retail|  74.0|         259.33|            74.0|    4.3

In [25]:
output_path = "data/FoodPriceData"

food_price_data.write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv(output_path)

# MODELLING

In [26]:
path = "data/FoodPriceData"
food_price_data = spark.read.csv(path, inferSchema=True, header=True)

In [27]:
food_price_data.show(5)

+----+-----+-------+--------+--------+------------------+-------------+-----+---------+------+---------------+----------------+--------------------+
|year|month| region|  county|  market|          category|    commodity| unit|pricetype| price|avg_rainfall_mm|normalized_price|log_normalized_price|
+----+-----+-------+--------+--------+------------------+-------------+-----+---------+------+---------------+----------------+--------------------+
|2014|    1|  Coast| Mombasa| Mombasa|cereals and tubers|        Maize|   KG|Wholesale| 38.44|         259.33|           38.44|  3.6747805297344347|
|2014|    1|  Coast| Mombasa| Mombasa|   pulses and nuts|        Beans|   KG|Wholesale| 79.99|         259.33|           79.99|  4.3943256902608985|
|2014|    1|  Coast| Mombasa| Mombasa|   pulses and nuts|  Beans (dry)|90 KG|Wholesale|5738.0|         259.33|           63.76|   4.170688128809434|
|2014|    1|Eastern|   Kitui|   Kitui|   pulses and nuts|  Beans (dry)|   KG|   Retail|  74.0|         259

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import PipelineModel

In [29]:
# Drop nulls from essential columns
model_data = food_price_data.dropna(subset=[
    "region", "county", "market", "category", "commodity", "unit", "pricetype",
    "log_normalized_price", "avg_rainfall_mm"
])


In [30]:
# Index categorical columns
categorical_cols = ["region", "county", "market", "category", "commodity", "unit", "pricetype"]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_indexed", handleInvalid="keep") for col in categorical_cols]

In [31]:
# Assemble features
feature_cols = [col+"_indexed" for col in categorical_cols] + ["month", "year", "avg_rainfall_mm"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [32]:
# Define model
dt = DecisionTreeRegressor(featuresCol="features", labelCol="log_normalized_price")

In [33]:
# Build pipeline
pipeline = Pipeline(stages=indexers + [assembler, dt])

In [34]:
# Split the data
train_data, test_data = model_data.randomSplit([0.8, 0.2], seed=42)

In [35]:
# Evaluator
evaluator = RegressionEvaluator(
    labelCol="log_normalized_price",
    predictionCol="prediction",
    metricName="rmse"
)

In [36]:
# Cross-validation with updated param grid
paramGrid = (
    ParamGridBuilder()
    .addGrid(dt.maxDepth, [5, 10])
    .addGrid(dt.maxBins, [100])
    .build()
)

cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3
)

In [37]:
# Train model with CV
cv_model = cv.fit(train_data)

In [38]:
# Predict
predictions = cv_model.transform(test_data)

In [39]:
# Evaluate
rmse = evaluator.evaluate(predictions)
print(f"Cross-Validated RMSE: {rmse:.2f}")

Cross-Validated RMSE: 0.24


In [40]:
# Save entire model pipeline
cv_model.bestModel.write().overwrite().save("model/log_price_pipeline_model")